<a href="https://colab.research.google.com/github/dclark18/H-SCricketMTA/blob/master/MTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import pandas as pd

url_dataset = 'https://raw.githubusercontent.com/dclark18/H-SCricketMTA/master/CricketMTA_sample.csv'
data = pd.read_csv(url_dataset, encoding = "ISO-8859-1")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46879 entries, 0 to 46878
Data columns (total 50 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Random                 46879 non-null  float64
 1   Date                   46879 non-null  object 
 2   Media Week             46879 non-null  object 
 3   Source                 46879 non-null  object 
 4   Video                  46879 non-null  object 
 5   Channel                46879 non-null  object 
 6   Tactic                 46879 non-null  object 
 7   Targeting              46879 non-null  object 
 8   Campaign               46879 non-null  object 
 9   Publisher              46879 non-null  object 
 10  Placement              46879 non-null  object 
 11  Creative               46879 non-null  object 
 12  Concept Name           67 non-null     object 
 13  Creative Name          67 non-null     object 
 14  Creative ID            31074 non-null  object 
 15  Ne

In [55]:
data.head()

,Random,Date,Media Week,Source,Video,Channel,Tactic,Targeting,Campaign,Publisher,Placement,Creative,Concept Name,Creative Name,Creative ID,New Campaign Name,New Creative Campaign,New Creative Concept,Phone Offer,Plan Offer,Search Segment,Brand,Screen,Media Type,Ad Size,Tactical Segment,Strategy,GRPs,Impressions,Clicks,Spend,100% Completion,Social Engagements,Store Visits,Total Device Orders,Order Value,MATCH,Customer Type,Order Status,Order From,Number Type,Gross Add,Event Row Count,Matched Event Count,iA,Landing Page Visits,Buyflow Entries,Google Foot Traffic,Campaign ID,Placement ID
0,0.743453,3/23/19,3/18/19,Neustar,N,Display,eComm,African American,I_ACW_CRI_AWA_022_Cricket Display_1H_2019,AppNexus,I_ACW_CRI_22_AppNexus_389_HTM_DTP_TND_RON_728x...,I_ACW_CRI_22_HTM_DTP_728x90_S_N_EN_NA_CRK-PRE-...,NaN,NaN,CRK0000223,Tax 2019Ê,Tax Promo,Pinwheel,Tax Free Samsung Amp Prime,No Plan Offer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0.00,0.0,0.0,0.0,0.0,NaN,No,NaN,NaN,NaN,NaN,0.0,1054.0,957.0,0.0,NaN,NaN,NaN,NaN,NaN
1,0.739026,3/19/19,3/18/19,Neustar,Y,Social,Upper Funnel,Hispanic,ATT-Cricket_Q1_USH_StoreVisits_FB_Feed_Native,facebook,L_ACW_CRI_013_FB_NEU_CRKT_UF_SLAB_One-Stop Sho...,L_ACW_CRI_013_Facebook_11_OLV_CRS_NA_ROS_1x1_N...,NaN,NaN,NaN,Tax 2019Ê,SLAB,Soccer,No Phone Offer,No Plan Offer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0.00,0.0,0.0,0.0,0.0,NaN,No,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,0.042701,3/30/19,3/25/19,4C,N,Social Local,Upper Funnel,African American,L_ACW_PSG_03_AWA_CRKT_FBIG_AA'19 Local Heavy U...,facebook,L_ACW_PSG_03_FBIG_CRM_CRKT_UF_Neustar Awarenes...,L_ACW_PSG_03_FBIG_HVD_NWF_Rose Glows 4/$100 + ...,NaN,NaN,NaN,Tax 2019Ê,AA Local Heavy Up,Rose Glows,Four Free Smartphones,4/$100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,0.02,NaN,NaN,0.0,0.0,0.0,No,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,0.459205,3/15/19,3/11/19,Neustar,Y,Upfront,Upper Funnel,General Market,I_ACW_APP_AWA_001_Cricket Scatter_Q1_2019,NBC,Not Applicable,I_ACW_CRI_21_GIF_DTP_728x90_S_N_EN_NA_CRK-PRE-...,NaN,NaN,CRK0000218,Tax 2019Ê,Tax Promo,Pinwheel,No Phone Offer,4/$100 + Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0.00,0.0,0.0,0.0,0.0,NaN,No,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,0.414004,2/26/19,2/25/19,Neustar,N,Display 1to1,eComm,General Market,I_ACW_CDM_001_AWA_Cricket Online Buyer_Q1_2019,AppNexus,I_ACW_CRI_1_AppNexus_18_BAN_DTP_TND_ROS_300x60...,I_ACW_CDM_1_HTM_DTP_300x600_S_N_EN_NA_CRK-PRE-...,NaN,NaN,NaN,Tax 2019Ê,Online Buyer,Online RTG,No Phone Offer,4/$100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0.00,0.0,0.0,0.0,0.0,NaN,No,NaN,NaN,NaN,NaN,0.0,2301.0,2123.0,0.0,0.0,0.0,0.0,NaN,NaN


#### Data Preparation

In [56]:
# Import all necessary libraries we will be using in our dataset

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import math
import re
import sklearn
import statistics
import random

from sklearn.feature_selection import SelectKBest, chi2, SelectPercentile, RFE, SelectFromModel

from sklearn.preprocessing import StandardScaler, Binarizer

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC

from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, auc, roc_curve
from IPython.display import display, HTML

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, TimeSeriesSplit, StratifiedShuffleSplit

from sklearn.naive_bayes import MultinomialNB

from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor,AdaBoostClassifier,RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

%matplotlib inline

matplotlib.style.use('ggplot')

import warnings
warnings.filterwarnings('ignore')
print(data.shape)

(46879, 50)


In [57]:
print (data.nunique())

Random                   46878
Date                       153
Media Week                  22
Source                       5
Video                        2
Channel                     10
Tactic                       3
Targeting                    3
Campaign                   134
Publisher                   42
Placement                 3789
Creative                   828
Concept Name                 3
Creative Name                6
Creative ID                 32
New Campaign Name            3
New Creative Campaign       19
New Creative Concept        62
Phone Offer                  9
Plan Offer                   9
Search Segment               1
Brand                        1
Screen                      13
Media Type                   9
Ad Size                     15
Tactical Segment             0
Strategy                     0
GRPs                         1
Impressions               5107
Clicks                     267
Spend                    12875
100% Completion           1631
Social E

In [58]:
print (data['Customer Type'].unique())

[nan 'UPGRADE' 'SIM_SWAP' 'NEW_CUST' 'ADDLINE']


Let's create a new column called CPIGA which will act as a aggregate of spend and the number of IAs garnered. We will ultimately be trying to classify against this value with our modeling. 

In [0]:
data['CPIGA'] = data['Spend']/data['iA']

In [0]:
# let's clean the dataset a little before moving on

# 1. Remove attributes that just arent useful for us
for col in ['Concept Name','Creative Name','Random', 'Search Segment', 
            'Brand', 'Tactical Segment', 'Strategy', 'GRPs', 'Customer', 
            'Order Status', 'Order From', 'Number Type', 'Google Foot Traffic']:
    if col in data:
        del data[col]

In [61]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46879 entries, 0 to 46878
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   46879 non-null  object 
 1   Media Week             46879 non-null  object 
 2   Source                 46879 non-null  object 
 3   Video                  46879 non-null  object 
 4   Channel                46879 non-null  object 
 5   Tactic                 46879 non-null  object 
 6   Targeting              46879 non-null  object 
 7   Campaign               46879 non-null  object 
 8   Publisher              46879 non-null  object 
 9   Placement              46879 non-null  object 
 10  Creative               46879 non-null  object 
 11  Creative ID            31074 non-null  object 
 12  New Campaign Name      46879 non-null  object 
 13  New Creative Campaign  46879 non-null  object 
 14  New Creative Concept   46879 non-null  object 
 15  Ph

### One Hot Encode Our categorical variables
We will be needing to one hot encode some of our categorical variables as they may provide some insight into our ia's that we generate. 

These varaibles will include the following:

* Source
* Video
* Channel
* Tactic
* Targeting
* New Campaign Name
* Phone Offer
* Plan Offer
* Match
* Customer Type
* Campaign Top

In [0]:
sourcedf = pd.get_dummies(data['Source'], prefix='source')
videodf = pd.get_dummies(data['Video'], prefix='video')
channeldf = pd.get_dummies(data['Channel'], prefix='channel')
tacticdf = pd.get_dummies(data['Tactic'], prefix='tactic')
targetingdf = pd.get_dummies(data['Targeting'], prefix='targeting')
cmpnamedf = pd.get_dummies(data['New Campaign Name'], prefix='camp_name')
phoneofferdf = pd.get_dummies(data['Phone Offer'], prefix='phone_offer')
planofferdf = pd.get_dummies(data['Plan Offer'], prefix='plan_offer')
matchdf = pd.get_dummies(data['MATCH'], prefix='match')
custtypedf = pd.get_dummies(data['Customer Type'], prefix='cust_type')
campaigniddf = pd.get_dummies(data['Campaign ID'], prefix='campaign_id')

In [63]:
sourcedf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46879 entries, 0 to 46878
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   source_4C                   46879 non-null  uint8
 1   source_DCM                  46879 non-null  uint8
 2   source_Innovid              46879 non-null  uint8
 3   source_Neustar              46879 non-null  uint8
 4   source_Twitter Ads Manager  46879 non-null  uint8
dtypes: uint8(5)
memory usage: 229.0 KB


In [0]:
data = pd.concat([data, sourcedf, videodf, channeldf, tacticdf, targetingdf, cmpnamedf, phoneofferdf, planofferdf, matchdf, custtypedf, campaigniddf])

In [65]:
data.head()

,Date,Media Week,Source,Video,Channel,Tactic,Targeting,Campaign,Publisher,Placement,Creative,Creative ID,New Campaign Name,New Creative Campaign,New Creative Concept,Phone Offer,Plan Offer,Screen,Media Type,Ad Size,Impressions,Clicks,Spend,100% Completion,Social Engagements,Store Visits,Total Device Orders,Order Value,MATCH,Customer Type,Gross Add,Event Row Count,Matched Event Count,iA,Landing Page Visits,Buyflow Entries,Campaign ID,Placement ID,CPIGA,source_4C,...,phone_offer_Tax Free Samsung Amp Prime,phone_offer_Tax Phone Selection,plan_offer_$100 Off,plan_offer_$30 Plans,plan_offer_$50 Plan,plan_offer_0.99,plan_offer_4/$100,plan_offer_4/$100 + Mexico,plan_offer_No Plan Offer,plan_offer_Unlimited + Mexico,plan_offer_Unlimited + Mexico and Canada,match_No,match_Yes,cust_type_ADDLINE,cust_type_NEW_CUST,cust_type_SIM_SWAP,cust_type_UPGRADE,campaign_id_22080394.0,campaign_id_22083908.0,campaign_id_22092970.0,campaign_id_22123914.0,campaign_id_22126087.0,campaign_id_22132698.0,campaign_id_22138168.0,campaign_id_22172931.0,campaign_id_22267583.0,campaign_id_22289204.0,campaign_id_22300694.0,campaign_id_22309146.0,campaign_id_22357961.0,campaign_id_22373571.0,campaign_id_22391019.0,campaign_id_22397066.0,campaign_id_22398760.0,campaign_id_22447767.0,campaign_id_22456671.0,campaign_id_22460409.0,campaign_id_22460922.0,campaign_id_22463025.0,campaign_id_22477724.0
0,3/23/19,3/18/19,Neustar,N,Display,eComm,African American,I_ACW_CRI_AWA_022_Cricket Display_1H_2019,AppNexus,I_ACW_CRI_22_AppNexus_389_HTM_DTP_TND_RON_728x...,I_ACW_CRI_22_HTM_DTP_728x90_S_N_EN_NA_CRK-PRE-...,CRK0000223,Tax 2019Ê,Tax Promo,Pinwheel,Tax Free Samsung Amp Prime,No Plan Offer,NaN,NaN,NaN,0.0,0,0.00,0.0,0.0,0.0,0.0,NaN,No,NaN,0.0,1054.0,957.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3/19/19,3/18/19,Neustar,Y,Social,Upper Funnel,Hispanic,ATT-Cricket_Q1_USH_StoreVisits_FB_Feed_Native,facebook,L_ACW_CRI_013_FB_NEU_CRKT_UF_SLAB_One-Stop Sho...,L_ACW_CRI_013_Facebook_11_OLV_CRS_NA_ROS_1x1_N...,NaN,Tax 2019Ê,SLAB,Soccer,No Phone Offer,No Plan Offer,NaN,NaN,NaN,0.0,0,0.00,0.0,0.0,0.0,0.0,NaN,No,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3/30/19,3/25/19,4C,N,Social Local,Upper Funnel,African American,L_ACW_PSG_03_AWA_CRKT_FBIG_AA'19 Local Heavy U...,facebook,L_ACW_PSG_03_FBIG_CRM_CRKT_UF_Neustar Awarenes...,L_ACW_PSG_03_FBIG_HVD_NWF_Rose Glows 4/$100 + ...,NaN,Tax 2019Ê,AA Local Heavy Up,Rose Glows,Four Free Smartphones,4/$100,NaN,NaN,NaN,1.0,NaN,0.02,NaN,NaN,0.0,0.0,0.0,No,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,inf,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3/15/19,3/11/19,Neustar,Y,Upfront,Upper Funnel,General Market,I_ACW_APP_AWA_001_Cricket Scatter_Q1_2019,NBC,Not Applicable,I_ACW_CRI_21_GIF_DTP_728x90_S_N_EN_NA_CRK-PRE-...,CRK0000218,Tax 2019Ê,Tax Promo,Pinwheel,No Phone Offer,4/$100 + Mexico,NaN,NaN,NaN,0.0,0,0.00,0.0,0.0,0.0,0.0,NaN,No,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2/26/19,2/25/19,Neustar,N,Display 1to1,eComm,General Market,I_ACW_CDM_001_AWA_Cricket Online Buyer_Q1_2019,AppNexus,I_ACW_CRI_1_AppNexus_18_BAN_DTP_TND_ROS_300x60...,I_ACW_CDM_1_HTM_DTP_300x600_S_N_EN_NA_CRK-PRE-...,NaN,Tax 2019Ê,Online Buyer,Online RTG,No Phone Offer,4/$100,NaN,NaN,NaN,0.0,0,0.00,0.0,0.0,0.0,0.0,NaN,No,NaN,0.0,2301.0,2123.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [66]:
# 2. Impute some missing values, grouped by their Pclass and SibSp numbers
df_grouped = data.groupby(by=['Placement ID']).sum()
print (df_grouped.describe())

        Impressions  ...  campaign_id_22477724.0
count  2.974000e+03  ...                  2974.0
mean   1.768607e+04  ...                     0.0
std    8.194946e+04  ...                     0.0
min    0.000000e+00  ...                     0.0
25%    2.800000e+01  ...                     0.0
50%    7.820000e+02  ...                     0.0
75%    6.539000e+03  ...                     0.0
max    1.442932e+06  ...                     0.0

[8 rows x 88 columns]


In [67]:
df_grouped.head()

,Impressions,Spend,100% Completion,Social Engagements,Store Visits,Total Device Orders,Order Value,Gross Add,Event Row Count,Matched Event Count,iA,Landing Page Visits,Buyflow Entries,Campaign ID,CPIGA,source_4C,source_DCM,source_Innovid,source_Neustar,source_Twitter Ads Manager,video_N,video_Y,channel_Audio,channel_Display,channel_Display 1to1,channel_Display Local,channel_OLV,channel_Search,channel_Social,channel_Social 1to1,channel_Social Local,channel_Upfront,tactic_Drive to Retail,tactic_Upper Funnel,tactic_eComm,targeting_African American,targeting_General Market,targeting_Hispanic,camp_name_New Year Phone Sale 2019,camp_name_Spring Promo 2019,...,phone_offer_Tax Free Samsung Amp Prime,phone_offer_Tax Phone Selection,plan_offer_$100 Off,plan_offer_$30 Plans,plan_offer_$50 Plan,plan_offer_0.99,plan_offer_4/$100,plan_offer_4/$100 + Mexico,plan_offer_No Plan Offer,plan_offer_Unlimited + Mexico,plan_offer_Unlimited + Mexico and Canada,match_No,match_Yes,cust_type_ADDLINE,cust_type_NEW_CUST,cust_type_SIM_SWAP,cust_type_UPGRADE,campaign_id_22080394.0,campaign_id_22083908.0,campaign_id_22092970.0,campaign_id_22123914.0,campaign_id_22126087.0,campaign_id_22132698.0,campaign_id_22138168.0,campaign_id_22172931.0,campaign_id_22267583.0,campaign_id_22289204.0,campaign_id_22300694.0,campaign_id_22309146.0,campaign_id_22357961.0,campaign_id_22373571.0,campaign_id_22391019.0,campaign_id_22397066.0,campaign_id_22398760.0,campaign_id_22447767.0,campaign_id_22456671.0,campaign_id_22460409.0,campaign_id_22460922.0,campaign_id_22463025.0,campaign_id_22477724.0
Placement ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
235648511.0,39291.0,609.243319,28655.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,227.0,0.0,949608044.0,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235648514.0,2253.0,31.270522,1814.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,485845976.0,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235648526.0,36739.0,555.565155,27548.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.0,0.0,419594252.0,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235648550.0,5951.0,87.787451,2046.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,419594252.0,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235648583.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22083908.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
# now use this grouping to fill the data set in each group, then transform back
df_imputed = df_grouped.transform(lambda grp: grp.fillna(grp.mean()))
df_imputed[['Impressions','Spend','100% Completion','Social Engagements', 
            'Store Visits','Total Device Orders', 'Order Value',
            'Gross Add', 'Event Row Count','Matched Event Count', 'iA', 'Landing Page Visits',
            'Buyflow Entries','CPIGA']] = df_grouped[['Impressions','Spend','100% Completion','Social Engagements', 
            'Store Visits','Total Device Orders', 'Order Value',
            'Gross Add', 'Event Row Count','Matched Event Count', 'iA', 'Landing Page Visits',
            'Buyflow Entries','CPIGA']]

print (df_imputed.info())                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

<class 'pandas.core.frame.DataFrame'>
Float64Index: 2974 entries, 235648511.0 to 247807212.0
Data columns (total 88 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Impressions                               2974 non-null   float64
 1   Spend                                     2974 non-null   float64
 2   100% Completion                           2974 non-null   float64
 3   Social Engagements                        2974 non-null   float64
 4   Store Visits                              2974 non-null   float64
 5   Total Device Orders                       2974 non-null   float64
 6   Order Value                               2974 non-null   float64
 7   Gross Add                                 2974 non-null   float64
 8   Event Row Count                           2974 non-null   float64
 9   Matched Event Count                       2974 non-null   float64
 10  iA               

In [0]:
df_imputed['CPIGA'] = df_imputed['Spend']/df_imputed['iA']

In [70]:
df_imputed.head()

,Impressions,Spend,100% Completion,Social Engagements,Store Visits,Total Device Orders,Order Value,Gross Add,Event Row Count,Matched Event Count,iA,Landing Page Visits,Buyflow Entries,Campaign ID,CPIGA,source_4C,source_DCM,source_Innovid,source_Neustar,source_Twitter Ads Manager,video_N,video_Y,channel_Audio,channel_Display,channel_Display 1to1,channel_Display Local,channel_OLV,channel_Search,channel_Social,channel_Social 1to1,channel_Social Local,channel_Upfront,tactic_Drive to Retail,tactic_Upper Funnel,tactic_eComm,targeting_African American,targeting_General Market,targeting_Hispanic,camp_name_New Year Phone Sale 2019,camp_name_Spring Promo 2019,...,phone_offer_Tax Free Samsung Amp Prime,phone_offer_Tax Phone Selection,plan_offer_$100 Off,plan_offer_$30 Plans,plan_offer_$50 Plan,plan_offer_0.99,plan_offer_4/$100,plan_offer_4/$100 + Mexico,plan_offer_No Plan Offer,plan_offer_Unlimited + Mexico,plan_offer_Unlimited + Mexico and Canada,match_No,match_Yes,cust_type_ADDLINE,cust_type_NEW_CUST,cust_type_SIM_SWAP,cust_type_UPGRADE,campaign_id_22080394.0,campaign_id_22083908.0,campaign_id_22092970.0,campaign_id_22123914.0,campaign_id_22126087.0,campaign_id_22132698.0,campaign_id_22138168.0,campaign_id_22172931.0,campaign_id_22267583.0,campaign_id_22289204.0,campaign_id_22300694.0,campaign_id_22309146.0,campaign_id_22357961.0,campaign_id_22373571.0,campaign_id_22391019.0,campaign_id_22397066.0,campaign_id_22398760.0,campaign_id_22447767.0,campaign_id_22456671.0,campaign_id_22460409.0,campaign_id_22460922.0,campaign_id_22463025.0,campaign_id_22477724.0
Placement ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
235648511.0,39291.0,609.243319,28655.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,227.0,0.0,949608044.0,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235648514.0,2253.0,31.270522,1814.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,485845976.0,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235648526.0,36739.0,555.565155,27548.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.0,0.0,419594252.0,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235648550.0,5951.0,87.787451,2046.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,419594252.0,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235648583.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22083908.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
df_imputed.describe()

,Impressions,Spend,100% Completion,Social Engagements,Store Visits,Total Device Orders,Order Value,Gross Add,Event Row Count,Matched Event Count,iA,Landing Page Visits,Buyflow Entries,Campaign ID,CPIGA,source_4C,source_DCM,source_Innovid,source_Neustar,source_Twitter Ads Manager,video_N,video_Y,channel_Audio,channel_Display,channel_Display 1to1,channel_Display Local,channel_OLV,channel_Search,channel_Social,channel_Social 1to1,channel_Social Local,channel_Upfront,tactic_Drive to Retail,tactic_Upper Funnel,tactic_eComm,targeting_African American,targeting_General Market,targeting_Hispanic,camp_name_New Year Phone Sale 2019,camp_name_Spring Promo 2019,...,phone_offer_Tax Free Samsung Amp Prime,phone_offer_Tax Phone Selection,plan_offer_$100 Off,plan_offer_$30 Plans,plan_offer_$50 Plan,plan_offer_0.99,plan_offer_4/$100,plan_offer_4/$100 + Mexico,plan_offer_No Plan Offer,plan_offer_Unlimited + Mexico,plan_offer_Unlimited + Mexico and Canada,match_No,match_Yes,cust_type_ADDLINE,cust_type_NEW_CUST,cust_type_SIM_SWAP,cust_type_UPGRADE,campaign_id_22080394.0,campaign_id_22083908.0,campaign_id_22092970.0,campaign_id_22123914.0,campaign_id_22126087.0,campaign_id_22132698.0,campaign_id_22138168.0,campaign_id_22172931.0,campaign_id_22267583.0,campaign_id_22289204.0,campaign_id_22300694.0,campaign_id_22309146.0,campaign_id_22357961.0,campaign_id_22373571.0,campaign_id_22391019.0,campaign_id_22397066.0,campaign_id_22398760.0,campaign_id_22447767.0,campaign_id_22456671.0,campaign_id_22460409.0,campaign_id_22460922.0,campaign_id_22463025.0,campaign_id_22477724.0
count,2.974000e+03,2974.000000,2.974000e+03,2974.0,2974.0,2974.000000,2.974000e+03,2974.000000,2974.0,2974.0,2974.0,2974.000000,2974.000000,2.974000e+03,2861.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,...,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0,2974.0
mean,1.768607e+04,175.382904,3.973492e+03,0.0,0.0,0.128110,1.391434e+07,0.055481,0.0,0.0,0.0,30.196705,0.592468,1.869158e+08,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,8.194946e+04,1007.507762,4.426512e+04,0.0,0.0,1.018859,1.041749e+08,0.521275,0.0,0.0,0.0,210.357206,3.321847,2.038799e+08,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.000000e+00,0.000000,0.000000e+00,0.0,0.0,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000,0.000000,2.208039e+07,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,2.800000e+01,0.211393,0.000000e+00,0.0,0.0,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000,0.000000,4.471592e+07,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,7.820000e+02,5.925529,0.000000e+00,0.0,0.0,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,1.000000,0.000000,1.324824e+08,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.

In [0]:
# 3. Remove attributes tha are coming up as zeros in our describe
for col in ['Social Engagements','Store Visits','Event Row Count','Matched Event Count', 'iA', 'CPIGA', 'Campaign ID']:
    if col in df_imputed:
        del df_imputed[col]

In [73]:
df_imputed.head()

,Impressions,Spend,100% Completion,Total Device Orders,Order Value,Gross Add,Landing Page Visits,Buyflow Entries,source_4C,source_DCM,source_Innovid,source_Neustar,source_Twitter Ads Manager,video_N,video_Y,channel_Audio,channel_Display,channel_Display 1to1,channel_Display Local,channel_OLV,channel_Search,channel_Social,channel_Social 1to1,channel_Social Local,channel_Upfront,tactic_Drive to Retail,tactic_Upper Funnel,tactic_eComm,targeting_African American,targeting_General Market,targeting_Hispanic,camp_name_New Year Phone Sale 2019,camp_name_Spring Promo 2019,camp_name_Tax 2019Ê,phone_offer_Apple iPhone,phone_offer_Four Free Smartphones,phone_offer_Generic,phone_offer_New Smartphone,phone_offer_No Phone Offer,phone_offer_Samsung Galaxy Halo,...,phone_offer_Tax Free Samsung Amp Prime,phone_offer_Tax Phone Selection,plan_offer_$100 Off,plan_offer_$30 Plans,plan_offer_$50 Plan,plan_offer_0.99,plan_offer_4/$100,plan_offer_4/$100 + Mexico,plan_offer_No Plan Offer,plan_offer_Unlimited + Mexico,plan_offer_Unlimited + Mexico and Canada,match_No,match_Yes,cust_type_ADDLINE,cust_type_NEW_CUST,cust_type_SIM_SWAP,cust_type_UPGRADE,campaign_id_22080394.0,campaign_id_22083908.0,campaign_id_22092970.0,campaign_id_22123914.0,campaign_id_22126087.0,campaign_id_22132698.0,campaign_id_22138168.0,campaign_id_22172931.0,campaign_id_22267583.0,campaign_id_22289204.0,campaign_id_22300694.0,campaign_id_22309146.0,campaign_id_22357961.0,campaign_id_22373571.0,campaign_id_22391019.0,campaign_id_22397066.0,campaign_id_22398760.0,campaign_id_22447767.0,campaign_id_22456671.0,campaign_id_22460409.0,campaign_id_22460922.0,campaign_id_22463025.0,campaign_id_22477724.0
Placement ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
235648511.0,39291.0,609.243319,28655.0,0.0,0.0,0.0,227.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235648514.0,2253.0,31.270522,1814.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235648526.0,36739.0,555.565155,27548.0,0.0,0.0,0.0,73.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235648550.0,5951.0,87.787451,2046.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235648583.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Drop:
Concept Name, Creative Name, Random, Search Segment, Brand, Tactical Segment, Strategy, GRPs, Customer, Order Status, Order From, Number Type, Google Foot Traffic



Possibly Drop: 

Creative ID, Screen, Ad Size (possibly encode nas as social, Media Type


One Hot Encode
Source
video
channel
Tactic
Targeting
New Campaign Name
Phone Offer
Plan Offer
MATCH

needs tinkering:

100% completion (need to account for non video placements), Ad Size (possibly encode nas as social), Media Type, 



In [0]:
sourcedf = pd.get_dummies(data['Source'], prefix='source')
videodf = pd.get_dummies(data['Video'], prefix='video')
channeldf = pd.get_dummies(data['Channel'], prefix='channel')
tacticdf = pd.get_dummies(data['Tactic'], prefix='tactic')
targetingdf = pd.get_dummies(data['Targeting'], prefix='targeting')
cmpnamedf = pd.get_dummies(data['New Campaign Name'], prefix='camp_name')
phoneofferdf = pd.get_dummies(data['Phone Offer'], prefix='phone_offer')
planofferdf = pd.get_dummies(data['Plan Offer'], prefix='plan_offer')
matchdf = pd.get_dummies(data['MATCH'], prefix='match')
custtypedf = pd.get_dummies(data['Customer Type'], prefix='cust_type')
campaigniddf = pd.get_dummies(data['Campaign ID'], prefix='campaign_id')

In [0]:
# Data drops 

# Remove unwanted columns, which include all the specific categories, 
# we will classify against the 
"""
cat_vars=['counter', 'liquor_category', 'store_parent',
 'month', 'year', 'monthyear', 'liquor_category_AMARETTO', 'liquor_category_BRANDY', 'liquor_category_GIN', 
 'liquor_category_LIQUEUR', 'liquor_category_Other', 'liquor_category_RUM', 'liquor_category_SCHNAPPS', 
 'liquor_category_TEQUILA', 'liquor_category_VODKA', 'month_Apr', 'month_Aug', 'month_Dec', 'month_Feb',
 'month_Jan', 'month_Jul', 'month_Jun', 'month_Mar', 'month_May', 'month_Nov', 'month_Oct', 'month_Sep', 
 'store_parent_CVS', 'store_parent_Caseys', 'store_parent_Hy-Vee', 'store_parent_Kum&Go', 
 'store_parent_Other', 'store_parent_QuikTrip', 'store_parent_SamsClub', 'store_parent_SmokingJoes', 
 'store_parent_Target', 'store_parent_Wal-Mart', 'store_parent_Walgreens']
data_vars=data.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]
"""
#keep our transformed detail, along with the timing and store name detail